In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import seaborn as sns
from math import floor, ceil
from pylab import rcParams

# Loading Keras packeges:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [10]:
# Loading daata:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

train_df.shape, test_df.shape

In [11]:
# Assining to X_train+CV and X_test, and seperate train to labels and data:
X_train_CV = train_df.drop("label", axis=1)
Y_train_CV_raw = train_df["label"]
X_test  = test_df
X_train_CV.shape, Y_train_CV_raw.shape, X_test.shape

In [12]:
# Converting Y_train to 0,1 format:
Y_train_CV = pd.DataFrame()

for i in range (0,10):
    Y_train_CV.loc[:, i] = pd.Series(((Y_train_CV_raw == i) *1)) 

Y_train_CV.head(10)

In [13]:
# Final input/output shapes:
X_train_CV.shape, Y_train_CV.shape, X_test.shape

In [14]:
# Deviding between train and CV sets:
train_cut = floor(X_train_CV.shape[0] * 0.85)
X_train = X_train_CV.iloc[0:train_cut].values
Y_train = Y_train_CV.iloc[0:train_cut].values
X_CV = X_train_CV.iloc[train_cut:].values
Y_CV = Y_train_CV.iloc[train_cut:].values

X_train.shape, Y_train.shape, X_CV.shape, Y_CV.shape

In [17]:
# Building CNN model:

model = Sequential()

# Reshape input layer:
model.add(Reshape((28, 28, 1), input_shape=(X_train.shape[1],)))
model.add(Conv2D(4, (2, 2), activation='relu', ))
model.add(Conv2D(8, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Train and fit the model:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=20, verbose=1, batch_size=32)

scores = model.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [18]:
# Evaluate model on CV set:
scores = model.evaluate(X_CV, Y_CV)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Predicting on test set:
Y_pred_raw = pd.DataFrame(model.predict(X_test, verbose=1, steps=None)).idxmax(axis=1)

In [ ]:
# Creating the output with the ImageId:

ImageId = pd.Series(np.arange(1, Y_pred_raw.size+1))
Y_pred = pd.DataFrame(Y_pred_raw, columns={'Label'})
Y_pred['ImageId'] = pd.Series(ImageId, index=Y_pred.index)


In [ ]:
# Outputing the CSV file:
submission = pd.DataFrame({
        "ImageId": Y_pred["ImageId"],
        "Label": Y_pred["Label"]
    })

submission.head(10)

submission.to_csv("submission.csv", index=False)